In [2]:
import carla #the sim library itself
client = carla.Client('localhost', 2000)
world = client.load_world('Town02')
map=world.get_map()


In [3]:
#global planner without u turn

import time
import numpy as np


map=world.get_map()
def key_exists(key, dict_of_dicts):
    for i in dict_of_dicts:
        if key in list(i.keys()):
            return True
    return False

waypoint_tuple_list = map.get_topology()
id_map = dict()
id_map_v = dict()
road_id_to_edge = dict()
graph={}
for waypoint in waypoint_tuple_list:
    x1, y1, x2, y2 = np.round([waypoint[0].transform.location.x, waypoint[0].transform.location.y,waypoint[1].transform.location.x, waypoint[1].transform.location.y], 0)
    road_id, section_id, lane_id = waypoint[0].road_id, waypoint[0].section_id, waypoint[0].lane_id
    if (x1,y1) not in id_map:
        id_map[(x1,y1)] = len(id_map)
        id_map_v[len(id_map_v)]=(x1,y1)
    if (x2,y2) not in id_map:
        id_map[(x2,y2)] = len(id_map)
        id_map_v[len(id_map_v)]=(x2,y2)
    if id_map[(x1,y1)] not in graph :
        graph[id_map[(x1,y1)]]={id_map[(x2,y2)]:waypoint[0]}
    else:
        if id_map[(x2,y2)] not in graph[id_map[(x1,y1)]]:
            graph[id_map[(x1,y1)]].update({id_map[(x2,y2)]:waypoint[0]})
    n1 = id_map[(x1,y1)]
    n2 = id_map[(x2,y2)]
    if road_id not in road_id_to_edge:
        road_id_to_edge[road_id] = dict()
    if section_id not in road_id_to_edge[road_id]:
        road_id_to_edge[road_id][section_id] = dict()
    road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)

def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def get_path(start,end):
    wa_start=map.get_waypoint(start)
    road_start_id, section_start_id, lane_start_id = wa_start.road_id, wa_start.section_id, wa_start.lane_id
    (n1_start, n2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id] 
    
    wa_end=map.get_waypoint(end)
    road_end_id, section_end_id, lane_end_id = wa_end.road_id, wa_end.section_id, wa_end.lane_id
    (n1_end, n2_end)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

    rout=[]
    if((n1_end, n2_end)!=(n1_start, n2_start)) or dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])<dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):
        (x1,y1),(x2,y2) = id_map_v[n2_start],id_map_v[n1_end]
        g = len(wa_start.next_until_lane_end(1))
        f=g+dist([x1,y1],[x2,y2])
        open = [[f,g,dist([x1,y1],[x2,y2]), x1, y1]]
        found = False 
        f_distance={node:float('inf') for node in graph}
        f_distance[n2_start]=f
    
        came_from={node:None for node in graph}
        came_from[n2_start]=n2_start
        #came_from={'s':None}
        #came_from={'su':None}
        #came_from={'n':None}
        #came_from={'nu':None}
        #came_from[n2_start]='s'
        
        while not found:
            if len(open) == 0:
                return "Fail"
                break
            else:
                min_ele = open[0][0]
                index=0
                for i in range(1, len(open)):
                    if open[i][0] < min_ele:
                        min_ele = open[i][0]
                        index=i
                next=open.pop(index)
                x = next[3]
                y = next[4]
                g = next[1]
                for i in graph[id_map[(x,y)]]:
                    (xf,yf)=id_map_v[i]
                    g2=len(graph[id_map[(x,y)]][i].next_until_lane_end(1))+g
                    f2=g2+dist([xf,yf],[x2,y2])
                    if came_from[i]==None:
                        open.append([f2,g2,dist([xf,yf],[x2,y2]), xf, yf])
                        f_distance[id_map[(xf,yf)]]=f2
                        came_from[id_map[(xf,yf)]]=id_map[(x,y)]
                        
                        if xf == x2 and yf == y2:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x2,y=y2,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)
                            while n1_end != n2_start:
                                n=came_from[n1_end]
                                ws=graph[n][n1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                n1_end=n
                            ws=map.get_waypoint(carla.Location(x=wa_start.transform.location.x,y=wa_start.transform.location.y,z=0)).next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break
    else:
        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
    return rout


In [4]:
#algorithm
import sys
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)
def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def astar():
    graph=[[i for i in range(12)]for j in range(12)]
    points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]
    start=[193,230]
    for i in range(12):
        for j in range(12):
            if i != j:
                #if (i==4 and j==3)or(i==6 and j==3)or(i==7 and j==0)or(i==8 and j==0)or(i==9 and j==0)or(i==11 and j==0):
                #    continue
                print(f"from {i} to {j}:")
                print(len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0))))
    f=0
    h=0
    g=0
    open=[[f,g,h,[start]]]
    path=[]
    while True:
        min_ele = open[0][0]
        index=0
        for i in range(1, len(open)):
            if open[i][0] < min_ele:
                min_ele = open[i][0]
                index=i
        next = open[index]
        open.pop(index)
        path=next[3].copy()
        for i in range(len(points)):
            if points[i] not in path:
                gp=len(get_path(carla.Location(x=path[len(path)-1][0],y=path[len(path)-1][1],z=0), carla.Location(x=points[i][0],y=points[i][1],z=0)))
                if (gp<450 and path[len(path)-1]==[95,305]) or(gp<300 and path[len(path)-1]!=[95,305]):
                    g=gp+next[1]
                    h=0
                    fpath=path.copy()
                    fpath.append(points[i])
                    for j in range(len(points)):
                        if points[j]not in fpath:
                            hj=dist(points[j],points[i])
                            #hj=len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0)))

                            if hj>h:
                                h=hj
                    f=2.1*h+g
                    open.append([f,g,h,fpath])
                    if len(fpath)==len(points)+1:
                        return fpath
astar()

from 0 to 1:
418
from 0 to 2:
312
from 0 to 3:
331
from 0 to 4:
320
from 0 to 5:
177
from 0 to 6:
259
from 0 to 7:
116
from 0 to 8:
197
from 0 to 9:
74
from 0 to 10:
263
from 0 to 11:
255
from 1 to 0:
136
from 1 to 2:
447
from 1 to 3:
466
from 1 to 4:
455
from 1 to 5:
312
from 1 to 6:
394
from 1 to 7:
251
from 1 to 8:
332
from 1 to 9:
209
from 1 to 10:
398
from 1 to 11:
390
from 2 to 0:
241
from 2 to 1:
106
from 2 to 3:
571
from 2 to 4:
560
from 2 to 5:
417
from 2 to 6:
499
from 2 to 7:
356
from 2 to 8:
437
from 2 to 9:
314
from 2 to 10:
503
from 2 to 11:
495
from 3 to 0:
318
from 3 to 1:
183
from 3 to 2:
77
from 3 to 4:
467
from 3 to 5:
236
from 3 to 6:
318
from 3 to 7:
175
from 3 to 8:
252
from 3 to 9:
391
from 3 to 10:
318
from 3 to 11:
310
from 4 to 0:
440
from 4 to 1:
305
from 4 to 2:
199
from 4 to 3:
401
from 4 to 5:
247
from 4 to 6:
58
from 4 to 7:
186
from 4 to 8:
173
from 4 to 9:
373
from 4 to 10:
239
from 4 to 11:
231
from 5 to 0:
463
from 5 to 1:
328
from 5 to 2:
222
from 5 

[[193, 230],
 [193, 200],
 [193, 125],
 [100, 103],
 [-8, 140],
 [20, 190],
 [40, 210],
 [80, 239],
 [135, 210],
 [165, 235],
 [40, 290],
 [95, 305],
 [2, 300]]

In [6]:
#add all routes together
#grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

ws1=get_path(carla.Location(x=193,y=230,z=0),carla.Location(x=193,y=200,z=0))
ws2=get_path(carla.Location(x=193,y=200,z=0),carla.Location(x=193,y=125,z=0))
ws3=get_path(carla.Location(x=193,y=125,z=0),carla.Location(x=100,y=103,z=0))
ws4=get_path(carla.Location(x=100,y=103,z=0),carla.Location(x=-8,y=140,z=0))
ws5=get_path(carla.Location(x=-8,y=140,z=0),carla.Location(x=20,y=190,z=0))
ws6=get_path(carla.Location(x=20,y=190,z=0),carla.Location(x=40,y=210,z=0))
ws7=get_path(carla.Location(x=40,y=210,z=0),carla.Location(x=80,y=239,z=0))
ws8=get_path(carla.Location(x=80,y=239,z=0),carla.Location(x=135,y=210,z=0))
ws9=get_path(carla.Location(x=135,y=210,z=0),carla.Location(x=40,y=290,z=0))
ws11=get_path(carla.Location(x=40,y=290,z=0),carla.Location(x=95,y=305,z=0))
ws12=get_path(carla.Location(x=95,y=305,z=0),carla.Location(x=165,y=235,z=0))
ws10=get_path(carla.Location(x=165,y=235,z=0),carla.Location(x=2,y=300,z=0))
wss=[ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8,ws9,ws10,ws11,ws12]
route=[]
for ws in wss:
    route.extend(ws)
    for w in ws:
        world.debug.draw_string(w.transform.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=600.0,
            persistent_lines=True)
print(len(route))

1297


In [22]:
#full code

#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla/')
from agents.navigation.global_route_planner import GlobalRoutePlanner

# connect to the sim 
#client = carla.Client('localhost', 2000)

# define speed contstants
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

def semantic_lidar_data(point_cloud_data,car,l):
    l.clear()
    for detection in point_cloud_data:
        #l.append([round(detection.point.x+car.get_location().x,1),round(detection.point.y+car.get_location().y,1)])
        l.append([detection.point.x+car.get_location().x,detection.point.y+car.get_location().y])

def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast')
    lidar_blueprint.set_attribute('channels', str(32))
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    lidar_blueprint.set_attribute("rotation_frequency",str(1000))
    lidar_blueprint.set_attribute("range",str(50))
    lidar_blueprint.set_attribute("upper_fov",str(1))
    lidar_blueprint.set_attribute("lower_fov",str(-1))
#    lidar_blueprint.set_attribute("horizontal_fov",str(360))
    return lidar_blueprint


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

#world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

start_point = spawn_points[0]
start_point.location.x=193
start_point.location.y=230
start_point.rotation.yaw=-90
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

get_blueprint_of_world = world.get_blueprint_library()
lidar_sensor = generate_lidar_blueprint(get_blueprint_of_world)
sensor_lidar_spawn_point = carla.Transform(carla.Location(x=0, y=0, z=1.2),
                                           carla.Rotation(pitch=0.000000, yaw=0.0, roll=0.000000))
#sensor = world.spawn_actor(lidar_sensor, sensor_lidar_spawn_point, attach_to=vehicle)

# create and show the navigation route like in Tutorial 3
#point_a = start_point.location #we start at where the car is
#sampling_resolution = 1
#grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
# now let' pick the longest possible route
#distance = 0
#for loc in spawn_points: # we start trying all spawn points 
                            #but we just exclude first at zero index
#    cur_route = grp.trace_route(point_a, loc.location)
#    if len(cur_route)>distance:
#        distance = len(cur_route)
#        route = cur_route
#draw the route in sim window - Note it does not get into the camera of the car
#for waypoint in route:
#    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
#        color=carla.Color(r=0, g=0, b=255), life_time=30.0,
#        persistent_lines=True)

#ladarData=[]
#sensor.listen(lambda point_cloud_data: semantic_lidar_data(point_cloud_data,vehicle,ladarData))
#l=set()

#main loop 
quit = False
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
while curr_wp<len(route)-1:

#    l=set(l)
#    ladarData1=ladarData.copy()
#    s={tuple(ladarData1[i]) for i in range(len(ladarData1))}
#    l=l.union(s)
#    l=list(l)
#    xf=len(l)
#    for i in range(xf):
#        if -l[xf-1-i][0]+vehicle.get_location().x>3 or abs(l[xf-1-i][1]-vehicle.get_location().y)>7:
#            l.pop(xf-1-i)
#    for i in l:
#        world.debug.draw_string(carla.Location(x=i[0],y=i[1],z=0), '^', draw_shadow=False,
#            color=carla.Color(r=0, g=0, b=255), life_time=1.0,
#            persistent_lines=True)
    # Carla Tick()
    world.tick()
    
    while curr_wp<len(route) and vehicle.get_transform().location.distance(route[curr_wp].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,route[curr_wp])
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

KeyboardInterrupt: 

In [23]:
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [85]:
l=set()
ladarData1=ladarData.copy()
s={tuple(ladarData1[i]) for i in range(len(ladarData1))}
l=l.union(s)
l=list(l)
xf=len(l)
#for i in range(xf):
#    if -l[xf-1-i][0]+vehicle.get_location().x>3 or abs(l[xf-1-i][1]-vehicle.get_location().y)>7:
#        l.pop(xf-1-i)
for i in l:
    world.debug.draw_string(carla.Location(x=i[0],y=i[1],z=0), '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=10.0,
        persistent_lines=True)

In [43]:
x=[(5* math.cos(vehicle.get_transform().rotation.yaw*math.pi/180)+3*-math.sin(vehicle.get_transform().rotation.yaw*math.pi/180)),(5*math.sin(vehicle.get_transform().rotation.yaw*math.pi/180)+3*math.cos(vehicle.get_transform().rotation.yaw*math.pi/180))]
print(x)

[-3.0000006657902447, 4.9999996005257925]


In [3]:

import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys
#astar()
client = carla.Client('localhost', 2000)
#world = client.get_world()
world = client.load_world('Town02')
map=world.get_map()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)

ws1=grp.trace_route(carla.Location(x=193,y=230,z=0),carla.Location(x=193,y=200,z=0))
ws2=grp.trace_route(carla.Location(x=193,y=200,z=0),carla.Location(x=193,y=125,z=0))
wa=map.get_waypoint(carla.Location(x=100,y=103,z=0))
ws3=grp.trace_route(carla.Location(x=193,y=125,z=0),wa.transform.location)
ws4=grp.trace_route(carla.Location(x=100,y=103,z=0),carla.Location(x=-8,y=140,z=0))
ws5=grp.trace_route(carla.Location(x=-8,y=140,z=0),carla.Location(x=20,y=190,z=0))
ws6=grp.trace_route(carla.Location(x=20,y=190,z=0),carla.Location(x=40,y=210,z=0))
ws7=grp.trace_route(carla.Location(x=40,y=210,z=0),carla.Location(x=80,y=239,z=0))
ws8=grp.trace_route(carla.Location(x=80,y=239,z=0),carla.Location(x=135,y=210,z=0))
wa=map.get_waypoint(carla.Location(x=165,y=235,z=0))
ws9=grp.trace_route(carla.Location(x=135,y=210,z=0),wa.transform.location)
ws10=grp.trace_route(carla.Location(x=165,y=235,z=0),carla.Location(x=40,y=290,z=0))
ws11=grp.trace_route(carla.Location(x=40,y=290,z=0),carla.Location(x=95,y=305,z=0))
ws12=grp.trace_route(carla.Location(x=95,y=305,z=0),carla.Location(x=2,y=300,z=0))
wss=[ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8,ws9,ws10,ws11,ws12]
world.debug.draw_string(carla.Location(x=191,y=200,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=191,y=125,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=100,y=101,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=-8,y=140,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=20,y=190,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=40,y=210,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=80,y=239,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=135,y=210,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=165,y=235,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=40,y=290,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=95,y=305,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)
world.debug.draw_string(carla.Location(x=2,y=300,z=5), '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=600.0,
        persistent_lines=True)

route=[]
for ws in wss:
    route.extend(ws)
    for w in ws:
        world.debug.draw_string(w[0].transform.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=600.0,
            persistent_lines=True)

# define speed contstants
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

#spawn_points = world.get_map().get_spawn_points()
start_point = route[0][0].transform
start_point.location.z=0.5
#spectator = world.get_spectator() #start_point was used to spawn the car but we move 1m up to avoid being on the floor
#spawn_points[0].location.y = 230
#spawn_points[0].location.x = 193
#spawn_points[0].rotation.pitch=-3
#spawn_points[0].rotation.yaw=3
#spawn_points[0].rotation.roll=-3
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*model3*')

vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
startTime=time.time()
while curr_wp<len(route)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    while curr_wp<len(route) and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,route[curr_wp][0])
    endTime=time.time()-startTime
    image = cv2.putText(image, 'Time: '+str(round(endTime,0)), org, font, fontScale, color, thickness, cv2.LINE_AA)
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    #image = cv2.putText(image, 'Next wp: '+str(curr_wp), org3, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75
    world.debug.draw_string(vehicle.get_location(), '^', draw_shadow=False,
            color=carla.Color(r=0, g=255, b=0), life_time=600.0,
            persistent_lines=True)
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)
    

#clean up
cv2.destroyAllWindows()
camera.stop()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()